In [ ]:
!pip install dedupe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.7/284.7 KB 8.6 MB/s eta 0:00:00


In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 5.8 MB/s eta 0:00:00


In [ ]:
import os
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode

In [ ]:
def preProcess(column):

  column = unidecode(column)
  column = re.sub('\n', ' ', column)
  column = re.sub('-', '', column)
  column = re.sub('/', ' ', column)
  column = re.sub("'", '', column)
  column = re.sub(",", '', column)
  column = re.sub(":", ' ', column)
  column = re.sub('  +', ' ', column)
  column = column.strip().strip('"').strip("'").lower().strip()
  if not column:
    column = None
  return column

def readData(filename):

  data_d = {}

  with open(filename,encoding='windows-1252') as f:
    reader = csv.DictReader(f)
    for i, row in enumerate(reader):
      clean_row = dict([(k, preProcess(v)) for (k, v) in row.items()])
      if clean_row['price']:
        clean_row['price'] = float(clean_row['price'][1:])
      data_d[clean_row['id']] = dict(clean_row)

  return data_d


In [ ]:
left_file = "/content/drive/MyDrive/RLTK/abt_test.csv"
right_file = "/content/drive/MyDrive/RLTK/buy_test.csv"

In [ ]:
print('importing data ...')
data_1 = readData(left_file)
data_2 = readData(right_file)

importing data ...


In [ ]:
data_1

{'34301': {'id': '34301',
  'name': 'lacie 1tb usb 2.0 external hard drive 301304u',
  'description': 'lacie 1tb usb 2.0 external hard drive 301304u up to 480mbits s (60mb s) interface transfer rate smart fanless cooling system for quiet operation blue led for ambient underside glow 8mb cache compatible with windows and mac',
  'price': None},
 '32546': {'id': '32546',
  'name': 'belkin hispeed usb 2.0 7port hub f5u237apls',
  'description': 'belkin hispeed usb 2.0 7port hub f5u237apls transfers data up to 480mbps installs easily with plugandplay convenience works seamlessly with all usb 1.1 and usb 2.0 devices overcurrent detection and safety mac and pc compatible silver finish',
  'price': 9.0},
 '31332': {'id': '31332',
  'name': 'sony universal remote control rmez4',
  'description': 'sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable erg

In [ ]:
def descriptions():
  for dataset in (data_1, data_2):
    for record in dataset.values():
      yield record['description']

In [ ]:
fields = [
            {'field': 'name', 'type': 'String'},
            {'field': 'name', 'type': 'Text', 'corpus': descriptions()},
            {'field': 'description', 'type': 'Text',
             'has missing': True, 'corpus': descriptions()},
            {'field': 'price', 'type': 'Price', 'has missing': True}]

In [ ]:
linker = dedupe.RecordLink(fields)

In [ ]:
# import time

# start_time = time.time()
# linker.prepare_training(data_1, data_2, sample_size=15000)
# print("Preparing Training:",(time.time()-start_time))

Preparing Training: 32.02925181388855


## Adding labelled Data

In [ ]:
import pandas as pd
gt = pd.read_csv("/content/drive/MyDrive/RLTK/labe_test.csv")

In [ ]:
train = gt

In [ ]:
train = gt.sample(frac = 0.70)

# Creating dataframe with
# rest of the 25% values
test = gt.drop(train.index)

In [ ]:
tot = list(data_1.keys())

In [ ]:
tot

['34301', '32546', '31332', '37807', '38477']

In [ ]:
import numpy as np
np.array(tot)

array(['34301', '32546', '31332', '37807', '38477'], dtype='<U5')

In [ ]:
# pos = gt['']
pos = train['idAbt'].astype('str').tolist()

In [ ]:
pos

['32546', '31332', '37807', '38477']

In [ ]:
import numpy as np
neg = np.setdiff1d(tot,pos)

In [ ]:
neg

array(['34301'], dtype='<U5')

In [ ]:

tot2 = list(data_2.keys())
pos2 = train['idBuy'].astype('str').tolist()
neg2 = np.setdiff1d(tot2,pos2)

In [ ]:
pos2, neg2

(['205761272', '205761272', '207682397', '10011646'],
 array(['202003159', '206615080'], dtype='<U9'))

In [ ]:
neg_1 = [(w,f) for w in neg for f in tot2]
neg_2 = [(f,w) for w in neg2 for f in tot]

In [ ]:
neg_1

[('34301', '206615080'),
 ('34301', '202003159'),
 ('34301', '205761272'),
 ('34301', '207682397'),
 ('34301', '10011646')]

In [ ]:
neg_2

[('34301', '202003159'),
 ('32546', '202003159'),
 ('31332', '202003159'),
 ('37807', '202003159'),
 ('38477', '202003159'),
 ('34301', '206615080'),
 ('32546', '206615080'),
 ('31332', '206615080'),
 ('37807', '206615080'),
 ('38477', '206615080')]

In [ ]:
neg_col = neg_1+neg_2

In [ ]:
def create_positive_data(train, positive_train):
  main_val = []
  for i,row in train.iterrows():
    inner = {}
    val = []
    abt = data_1[str(row['idAbt'])]
    val.append(abt)
    buy = data_2[str(row['idBuy'])]
    val.append(buy)

    inner["__class__"] = "tuple"
    inner["__value__"] = val
    main_val.append(inner)
  positive_train["match"]=main_val
  return positive_train




In [ ]:
import random
def create_negative_data(positive_train, neg_col):

  filtered_pair = random.sample(neg_col, 5)
  main_val = []
  for i,j in filtered_pair:
    inner = {}
    val = []
    abt = data_1[str(i)]
    val.append(abt)
    buy = data_2[str(j)]
    val.append(buy)

    inner["__class__"] = "tuple"
    inner["__value__"] = val
    main_val.append(inner)
  positive_train["distinct"]=main_val
  return positive_train

In [ ]:
positive_train = {}
final_train = create_negative_data(positive_train,neg_col)

In [ ]:
final_json

{'distinct': [{'__class__': 'tuple',
   '__value__': [{'id': '31332',
     'name': 'sony universal remote control rmez4',
     'description': 'sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable ergonomic design silver finish',
     'price': 6.0},
    {'id': '202003159',
     'name': 'belkin hispeed usb 2.0 7port hub f5u237apls',
     'description': 'usb 2.0 7port hub apple #tb857ll a',
     'manufacturer': 'belkin',
     'price': None}]},
  {'__class__': 'tuple',
   '__value__': [{'id': '37807',
     'name': 'logitech black v450 nano cordless laser mouse for notebooks 910000619',
     'description': 'logitech black v450 nano cordless laser mouse for notebooks 910000619 laser precision sidetoside scrolling plus zoom advanced 2.4 ghz wireless connection storable nano reciever 12 months battery life with indicator light black finish',
     'pric

In [ ]:
import json

with open('/content/drive/MyDrive/RLTK/original_train_file_test_new', 'w') as convert_file:
  convert_file.write(json.dumps(final_json))

In [ ]:
len(final_json['distinct']),len(final_json['match'])

(2304, 768)

In [ ]:
final_json

{'distinct': [{'__class__': 'tuple',
   '__value__': [{'id': '28205',
     'name': 'panasonic kxtg6702b 5.8 ghz fhss gigarange expandable black cordless phone system kxtg6702b',
     'description': 'panasonic kxtg6702b 5.8 ghz fhss gigarange expandable black cordless phone system kxtg6702b alldigital answering system lcd call counter speakerphone navigator key up to 8 handsets with just one phone jack line status indicator voice scramble handset locator volume control black finish',
     'price': 197.0},
    {'id': '209656971',
     'name': 'apple ipod nano 16gb flash portable media player mb913ll a',
     'description': 'audio player video player photo viewer 2 color lcd 16gb flash memory green',
     'manufacturer': 'apple',
     'price': 178.99}]},
  {'__class__': 'tuple',
   '__value__': [{'id': '16669',
     'name': 'omnimount tv top shelf mount cch1b',
     'description': 'omnimount tv top shelf mount cch1b tv top shelf mount for crt lcd & dlp tvs black finish (photo showing plan

In [ ]:
# dedupe.console_label(linker)

In [ ]:
# with open("/content/drive/MyDrive/RLTK/training_file", 'w') as tf:
#   linker.write_training(tf)

# Test labelling part

In [ ]:
abt = [
{"id": "34301", "name": "lacie 1tb usb 2.0 external hard drive 301304u", "description": "lacie 1tb usb 2.0 external hard drive 301304u up to 480mbits s (60mb s) interface transfer rate smart fanless cooling system for quiet operation blue led for ambient underside glow 8mb cache compatible with windows and mac"},
{"id": "32546", "name": "belkin hispeed usb 2.0 7port hub f5u237apls", "description": "belkin hispeed usb 2.0 7port hub f5u237apls transfers data up to 480mbps installs easily with plugandplay convenience works seamlessly with all usb 1.1 and usb 2.0 devices overcurrent detection and safety mac and pc compatible silver finish", "price": 49.0},
{"id": "31332", "name": "sony universal remote control rmez4", "description": "sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable ergonomic design silver finish", "price": 16.0},
{"id": "37807", "name": "logitech black v450 nano cordless laser mouse for notebooks 910000619", "description": "logitech black v450 nano cordless laser mouse for notebooks 910000619 laser precision sidetoside scrolling plus zoom advanced 2.4 ghz wireless connection storable nano reciever 12 months battery life with indicator light black finish", "price": 49.0},
{'id': '38477', 'name': 'linksys etherfast 8port 10 100 switch ezxs88w','description': 'linksys etherfast 8port 10 100 switch ezxs88w 10 100 dualspeed perport perfect for optimizing 10baset and 100basetx hardware on the same network speeds of up to 200mbps in full duplex operation eliminate bandwidth constraints and clear up bottlenecks','price': 44.0}
]

buy= [
  {"id": "206615080", "name": "lacie usb 2.0 external hard drive 301304u", "description": "lacie 1tb usb 2.0 7200rpm desktop external hard drive design by neil poulton", "manufacturer": "lacie"},
  {"id": "202003159", "name": "belkin hispeed usb 2.0 7port hub f5u237apls", "description": "usb 2.0 7port hub apple #tb857ll a", "manufacturer": "belkin"},
  {"id": "205761272", "name": "sony universal remote control rmez4", "description": "tv cable box universal remote", "manufacturer": "sony", "price": 12.72},
  {"id": "207682397", "name": "logitech v450 nano cordless laser mouse for notebooks 910000619", "description": "laser", "manufacturer": "logitech"},
  {'id': '10011646', 'name': 'linksys etherfast ezxs88w ethernet switch ezxs88w','description': 'linksys etherfast 8port 10 100 switch (new workgroup)','manufacturer': 'linksys','price': None}
  ]


In [ ]:
import pandas as pd

abt_df = pd.DataFrame(abt)
buy_df = pd.DataFrame(buy)

In [ ]:
abt_df

,id,name,description,price
0,34301,lacie 1tb usb 2.0 external hard drive 301304u,lacie 1tb usb 2.0 external hard drive 301304u ...,NaN
1,32546,belkin hispeed usb 2.0 7port hub f5u237apls,belkin hispeed usb 2.0 7port hub f5u237apls tr...,49.0
2,31332,sony universal remote control rmez4,sony universal remote control rmez4 easytouse ...,16.0
3,37807,logitech black v450 nano cordless laser mouse ...,logitech black v450 nano cordless laser mouse ...,49.0
4,38477,linksys etherfast 8port 10 100 switch ezxs88w,linksys etherfast 8port 10 100 switch ezxs88w ...,44.0


In [ ]:
abt_df.to_csv("/content/drive/MyDrive/RLTK/abt_test.csv",index=None)

In [ ]:
buy_df.to_csv("/content/drive/MyDrive/RLTK/buy_test.csv",index=None)

In [ ]:
abt_df['id']

0    34301
1    32546
2    31332
3    37807
4    38477
Name: id, dtype: object

In [ ]:
labe = [
    {"idAbt":"32546",
     "idBuy":"205761272"},
     {
      "idAbt":"31332",
      "idBuy":"205761272"
     },
     {
      "idAbt":"37807",
      "idBuy":"207682397"
     },
     {
      "idAbt":"38477",
      "idBuy":"10011646"
     }
]

In [ ]:
labe_df = pd.DataFrame(labe)

In [ ]:
[(('31332', '205761272'), 0.59807307),
 (('32546', '202003159'), 0.56793463),
 (('37807', '207682397'), 0.548983),
 (('34301', '206615080'), 0.5461112),
 (('38477', '10011646'), 0.48580316)]

In [ ]:
labe_df

,idAbt,idBuy
0,32546,205761272
1,31332,205761272
2,37807,207682397
3,38477,10011646


In [ ]:
labe_df.to_csv("/content/drive/MyDrive/RLTK/labe_test.csv",index=None)

In [ ]:
data_1 = {}
for i,clean_row in abt_df.iterrows():
  if clean_row['price']:
    clean_row['price'] = float(clean_row['price'])
  data_1[clean_row['id']] = dict(clean_row)

In [ ]:
data_1

{'34301': {'id': '34301',
  'name': 'lacie 1tb usb 2.0 external hard drive 301304u',
  'description': 'lacie 1tb usb 2.0 external hard drive 301304u up to 480mbits s (60mb s) interface transfer rate smart fanless cooling system for quiet operation blue led for ambient underside glow 8mb cache compatible with windows and mac',
  'price': nan},
 '32546': {'id': '32546',
  'name': 'belkin hispeed usb 2.0 7port hub f5u237apls',
  'description': 'belkin hispeed usb 2.0 7port hub f5u237apls transfers data up to 480mbps installs easily with plugandplay convenience works seamlessly with all usb 1.1 and usb 2.0 devices overcurrent detection and safety mac and pc compatible silver finish',
  'price': 49.0},
 '31332': {'id': '31332',
  'name': 'sony universal remote control rmez4',
  'description': 'sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable erg

In [ ]:
{
    "distinct": [
        {
            "__class__": "tuple",
         "__value__": [
             {"id": "34301", "name": "lacie 1tb usb 2.0 external hard drive 301304u", "description": "lacie 1tb usb 2.0 external hard drive 301304u up to 480mbits s (60mb s) interface transfer rate smart fanless cooling system for quiet operation blue led for ambient underside glow 8mb cache compatible with windows and mac", "price": null},
              {"id": "206615080", "name": "lacie usb 2.0 external hard drive 301304u", "description": "lacie 1tb usb 2.0 7200rpm desktop external hard drive design by neil poulton", "manufacturer": "lacie", "price": null}
              ]
         }
         ],
  "match": [
      {"__class__": "tuple",
       "__value__": [
           {"id": "32546", "name": "belkin hispeed usb 2.0 7port hub f5u237apls", "description": "belkin hispeed usb 2.0 7port hub f5u237apls transfers data up to 480mbps installs easily with plugandplay convenience works seamlessly with all usb 1.1 and usb 2.0 devices overcurrent detection and safety mac and pc compatible silver finish", "price": 49.0},
           {"id": "202003159", "name": "belkin hispeed usb 2.0 7port hub f5u237apls", "description": "usb 2.0 7port hub apple #tb857ll a", "manufacturer": "belkin", "price": null}
           ]
       },
            {"__class__": "tuple",
             "__value__": [
                 {"id": "31332", "name": "sony universal remote control rmez4", "description": "sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable ergonomic design silver finish", "price": 16.0},
                 {"id": "205761272", "name": "sony universal remote control rmez4", "description": "tv cable box universal remote", "manufacturer": "sony", "price": 12.72}
                 ]
             },
            {"__class__": "tuple",
             "__value__": [
                 {"id": "37807", "name": "logitech black v450 nano cordless laser mouse for notebooks 910000619", "description": "logitech black v450 nano cordless laser mouse for notebooks 910000619 laser precision sidetoside scrolling plus zoom advanced 2.4 ghz wireless connection storable nano reciever 12 months battery life with indicator light black finish", "price": 49.0},
                  {"id": "207682397", "name": "logitech v450 nano cordless laser mouse for notebooks 910000619", "description": "laser", "manufacturer": "logitech", "price": null}
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              ]}]}

In [ ]:
import time
start_time = time.time()

with open("/content/drive/MyDrive/RLTK/original_train_file_test_new") as tf:

  linker.prepare_training(data_1,
                        data_2,
                        training_file=tf,
                        sample_size=15000)
print("Preparing Training:",(time.time()-start_time))

Preparing Training: 0.4032120704650879


In [ ]:
start_time = time.time()

linker.train()
print("Preparing Training:",(time.time()-start_time))

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Preparing Training: 5.312023401260376


In [ ]:
pair = linker.pairs(data_1,data_2)

In [ ]:
data_1

{'34301': {'id': '34301',
  'name': 'lacie 1tb usb 2.0 external hard drive 301304u',
  'description': 'lacie 1tb usb 2.0 external hard drive 301304u up to 480mbits s (60mb s) interface transfer rate smart fanless cooling system for quiet operation blue led for ambient underside glow 8mb cache compatible with windows and mac',
  'price': None},
 '32546': {'id': '32546',
  'name': 'belkin hispeed usb 2.0 7port hub f5u237apls',
  'description': 'belkin hispeed usb 2.0 7port hub f5u237apls transfers data up to 480mbps installs easily with plugandplay convenience works seamlessly with all usb 1.1 and usb 2.0 devices overcurrent detection and safety mac and pc compatible silver finish',
  'price': 9.0},
 '31332': {'id': '31332',
  'name': 'sony universal remote control rmez4',
  'description': 'sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable erg

In [ ]:
list(pair)

[(('31332',
   {'id': '31332',
    'name': 'sony universal remote control rmez4',
    'description': 'sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable ergonomic design silver finish',
    'price': 6.0}),
  ('205761272',
   {'id': '205761272',
    'name': 'sony universal remote control rmez4',
    'description': 'tv cable box universal remote',
    'manufacturer': 'sony',
    'price': 2.72})),
 (('31332',
   {'id': '31332',
    'name': 'sony universal remote control rmez4',
    'description': 'sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable ergonomic design silver finish',
    'price': 6.0}),
  ('202003159',
   {'id': '202003159',
    'name': 'belkin hispeed usb 2.0 7port hub f5u237apls',
    'd

In [ ]:
linked_records = linker.join(data_1, data_2, 0.3)


In [ ]:
linked_records

[(('31332', '205761272'), 0.59807307),
 (('32546', '202003159'), 0.56793463),
 (('37807', '207682397'), 0.548983),
 (('34301', '206615080'), 0.5461112),
 (('38477', '10011646'), 0.48580316)]

In [ ]:
linker.uncertain_pairs()

[({'id': '34947',
   'name': 'lg lfc25770st 25.0 cu. ft. stainless steel french door bottom freezer refrigerator lfc25770ss',
   'description': 'lg lfc25770st 25.0 cu. ft. stainless steel french door bottom freezer refrigerator lfc25770ss 4 slide out tempered glass shelves internal led touch pad digital temperature controls 2 humidity crispers multiair flow cooling iceplus accelerated freezing function lodecibel quiet operation bioshield antibacterial door seal pullout freezer drawer stainless steel finish',
   'price': None},
  {'id': '208117920',
   'name': 'lg dishwasher',
   'description': None,
   'manufacturer': 'lg electronics',
   'price': None})]

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/RLTK/abt_test.csv")
dff = pd.read_csv("/content/drive/MyDrive/RLTK/buy_test.csv")


In [ ]:
import json
with open("/content/drive/MyDrive/sec/mydata.json", "w") as final:
  for i, v in df.iterrows():
    print(v.to_json())
    # Serializing json
    json_object = json.dumps(v.to_json(), indent=4)
    json.dump(json_object, final)

{"id":34301,"name":"lacie 1tb usb 2.0 external hard drive 301304u","description":"lacie 1tb usb 2.0 external hard drive 301304u up to 480mbits s (60mb s) interface transfer rate smart fanless cooling system for quiet operation blue led for ambient underside glow 8mb cache compatible with windows and mac","price":null}
{"id":32546,"name":"belkin hispeed usb 2.0 7port hub f5u237apls","description":"belkin hispeed usb 2.0 7port hub f5u237apls transfers data up to 480mbps installs easily with plugandplay convenience works seamlessly with all usb 1.1 and usb 2.0 devices overcurrent detection and safety mac and pc compatible silver finish","price":49.0}
{"id":31332,"name":"sony universal remote control rmez4","description":"sony universal remote control rmez4 easytouse simplified functions controls tv and cable box compatible with most of major brands large buttons for easy operation 3minute memory backup comfortable ergonomic design silver finish","price":16.0}
{"id":37807,"name":"logitech 

In [ ]:
for i, v in dff.iterrows():
    print(v.to_json())

{"id":206615080,"name":"lacie usb 2.0 external hard drive 301304u","description":"lacie 1tb usb 2.0 7200rpm desktop external hard drive design by neil poulton","manufacturer":"lacie","price":null}
{"id":202003159,"name":"belkin hispeed usb 2.0 7port hub f5u237apls","description":"usb 2.0 7port hub apple #tb857ll a","manufacturer":"belkin","price":null}
{"id":205761272,"name":"sony universal remote control rmez4","description":"tv cable box universal remote","manufacturer":"sony","price":12.72}
{"id":207682397,"name":"logitech v450 nano cordless laser mouse for notebooks 910000619","description":"laser","manufacturer":"logitech","price":null}
{"id":10011646,"name":"linksys etherfast ezxs88w ethernet switch ezxs88w","description":"linksys etherfast 8port 10 100 switch (new workgroup)","manufacturer":"linksys","price":null}
